In [79]:
import pandas as pd
import json
import collections
import scipy.io
import numpy as np
import mpu

In [92]:
bus = scipy.io.loadmat('MATPOWER/bus.mat')['bus']
power_demand = bus[:, 2]
df = pd.read_csv("GIS/CATS_gens.csv")
df = df[df['Pmax'] != 0.0].to_numpy()
#df = df.drop_duplicates(subset=['PlantCode','GenID']).to_numpy()

In [82]:
gen_emission = pd.read_excel("Book1.xlsx")
power_plants = pd.read_csv("Power_Plants_in_the_U.S..csv")
gen_emission = gen_emission[gen_emission['Plant state abbreviation'] == 'CA'].dropna()
gen_emission = gen_emission.rename(columns={'DOE/EIA ORIS plant or facility code': 'Plant_Code'})
gen_emission = power_plants.merge(gen_emission, on='Plant_Code', how='inner')
gen_emission = gen_emission.drop(['Plant latitude', 'Plant longitude', 'Plant state abbreviation', 'Plant name', 'State'], axis=1)
gen_emission.groupby(['PrimSource'])['Plant annual CO2 total output emission rate (lb/MWh)'].mean()

PrimSource
batteries            0.000000
biomass             76.203671
coal              1149.018000
geothermal         166.657414
hydroelectric        0.000000
natural gas       1255.016816
other              884.041200
petroleum         1808.541000
pumped storage       0.000000
solar              144.994400
Name: Plant annual CO2 total output emission rate (lb/MWh), dtype: float64

In [83]:
#gen_emission = gen_emission.round(6)
df = df.rename(columns={"PlantCode": "Plant_Code"})
df = df.merge(gen_emission, on=["Plant_Code"], how="inner")
df = df.drop(['GenID','Qmax','Qmin','X','Y','Plant_Name','Qg','Zip'], axis=1)
df = df.drop_duplicates(['Plant_Code','FuelType'])
df.groupby(['FuelType'])['Plant annual CO2 total output emission rate (lb/MWh)'].mean()

FuelType
All Other                                  629.326000
Batteries                                  328.980765
Conventional Hydroelectric                   0.000000
Conventional Steam Coal                   1149.018000
Geothermal                                 169.581263
Hydroelectric Pumped Storage                 0.000000
Landfill Gas                                70.810367
Municipal Solid Waste                     2369.855000
Natural Gas Fired Combined Cycle           914.125089
Natural Gas Fired Combustion Turbine      1124.875640
Natural Gas Internal Combustion Engine     979.600500
Natural Gas Steam Turbine                 5969.755778
Other Gases                               1036.301571
Other Natural Gas                          618.136000
Other Waste Biomass                         80.213905
Petroleum Liquids                         2042.054000
Solar Photovoltaic                         608.258200
Solar Thermal without Energy Storage         4.692000
Wood/Wood Waste Bio

In [93]:
type_to_emission = collections.defaultdict(float)
type_to_emission['Conventional Hydroelectric'] = 0
type_to_emission['Hydroelectric Pumped Storage'] = 0
type_to_emission['Petroleum Liquids'] = 2042.05
type_to_emission['Natural Gas Internal Combustion Engine'] = 979.6
type_to_emission['Natural Gas Fired Combined Cycle'] = 914.13
type_to_emission['Natural Gas Steam Turbine'] = 5969.76
type_to_emission['Natural Gas Fired Combustion Turbine'] = 1124.88
#type_to_emission['Nuclear'] = 0
type_to_emission['Geothermal'] = 169.58
#type_to_emission['Onshore Wind Turbine'] = 0
type_to_emission['Other Waste Biomass'] = 80.21
type_to_emission['Wood/Wood Waste Biomass'] = 7.53
type_to_emission['Landfill Gas'] = 70.81
type_to_emission['Solar Photovoltaic'] = 608.26
type_to_emission['Solar Thermal without Energy Storage'] = 4.69
type_to_emission['Conventional Steam Coal'] = 1149.02
type_to_emission['Other Gases'] = 1036.3
type_to_emission['Batteries'] = 328.98
#type_to_emission['Petroleum Coke'] = 1808.5
type_to_emission['Municipal Solid Waste'] = 2369.86
type_to_emission['Other Natural Gas'] = 618.14
type_to_emission['All Other'] = 629.33
#type_to_emission['IMPORT'] = 884
#type_to_emission['Synchronous Condenser'] = 884

In [94]:
branch_ = scipy.io.loadmat('MATPOWER/branch.mat')['brach']
branch_from_bus = list(map(int, branch_[:, 0]-1))
branch_to_bus = list(map(int, branch_[:, 1]-1))
connection_all = branch_[:, 0:2]-1

In [95]:
f = open("pf_solution.json")
sol = json.load(f)
gen = [0]*2149
gen_cost = [0]*2149
power_generation = [0]*2149
carbon_emission = [0]*2149
branch_power_to = [0]*10823
branch_power_from = [0]*10823
for line, val in sol['solution']['gen'].items():
    if val['pg'] != 0.0:
        gen[int(line)-1] = df[int(line)-1][2]-1
        carbon_emission[int(line)-1] = type_to_emission[df[int(line)-1][3]]
        gen_cost[int(line)-1] = val['pg_cost']*100
        power_generation[int(line)-1] = val['pg']*100

for line, val in sol['solution']['branch'].items():
    branch_power_from[int(line)-1] = val['pf']*100
    branch_power_to[int(line)-1] = val['pt']*100

f.close()

In [96]:
graph = collections.defaultdict(list) # from: (to, line)
graph_reverse = collections.defaultdict(list) # to: (from, line)
branch_power_flow = collections.defaultdict(float)
for i in range(10823):
    if branch_power_to[i] > 0:
        graph[branch_from_bus[i]].append((branch_to_bus[i], i))
        graph_reverse[branch_to_bus[i]].append((branch_from_bus[i], i))
    else:
        graph[branch_to_bus[i]].append((branch_from_bus[i], i))
        graph_reverse[branch_from_bus[i]].append((branch_to_bus[i], i))
    branch_power_flow[i] = abs(branch_power_to[i])

In [97]:
num_bus = len(bus)
num_branch = len(branch_from_bus)
num_gen = len(gen)
print("Number of transmission lines and transformers: " + str(num_branch))
print("Number of buses: " + str(num_bus))
print("Number of generators: " + str(num_gen))

Number of transmission lines and transformers: 10823
Number of buses: 8870
Number of generators: 2149


In [98]:
line_to_gen = collections.defaultdict(set)
node_to_gen = collections.defaultdict(set)
def dfs(g, n, visited):
    if n in visited: return
    visited.add(n)
    node_to_gen[n].add(g)
    for nei, line in graph[n]:
        line_to_gen[line].add(g)
        dfs(g, nei, visited)
for i, g in enumerate(gen):
    visited = set()
    dfs(g, g, visited)

In [99]:
line_prop_mat=np.zeros((num_gen, num_branch), dtype=float)
bus_prop_mat=np.zeros((num_gen, num_bus), dtype=float)
# initially, if node_to_gen only has 1 generator, bus_prop_mat start with 1
for k, v in node_to_gen.items():
    if len(v) == 1:
        idx = list(gen).index(list(v)[0])
        bus_prop_mat[idx][int(k)] = 1.0
for k, v in line_to_gen.items():
    if len(v) == 1:
        idx = list(gen).index(list(v)[0])
        line_prop_mat[idx][int(k)] = 1.0

In [132]:
power_generation[1598]

37.49473149033279

In [100]:
sum(power_generation)

44008.91585859414

In [107]:
# visit nodes in topological order
# step 1: calculate bus_prop based on in flowing lines_prop
# step 2: calculate line_prop of out flowing lines
in_degree = collections.defaultdict(int)
for i, v in graph_reverse.items():
    in_degree[i] = len(v)
q = [] # list of nodes with no inflow
topo_order = []
for g in gen:
    if in_degree[g] == 0:
        q.append(g)
while q:
    cur = int(q.pop(0))
    if len(node_to_gen[cur]) > 1:
        out_total = power_demand[cur]
        for nei, out_line in graph[cur]:
            out_total += branch_power_flow[out_line]
        
        for g in node_to_gen[cur]:
            idx = list(gen).index(g)
            if cur == g:
                if out_total != 0.0: 
                    bus_prop_mat[idx][cur] = power_generation[idx]/out_total
            else:
                for nei, in_line in graph_reverse[cur]:
                    if out_total != 0.0: 
                        bus_prop_mat[idx][cur] += branch_power_flow[in_line]*line_prop_mat[idx][in_line]/out_total
        for g in node_to_gen[cur]:
            idx = list(gen).index(g)
            for nei, out_line in graph[cur]:
                line_prop_mat[idx][out_line] = bus_prop_mat[idx][cur]

    topo_order.append(cur)
    for nei, line in graph[cur]:
        in_degree[nei] -= 1
        if in_degree[nei] == 0:
            q.append(nei)

In [102]:
print(graph)

defaultdict(<class 'list'>, {0: [(2409, 0)], 19: [(2409, 1), (3167, 1631), (3167, 1634), (3170, 1638), (3170, 1641)], 2409: [(2410, 2)], 1617: [(2410, 3)], 2410: [(1616, 4)], 1616: [(2411, 5)], 2359: [(2358, 6)], 2030: [(2425, 7), (2426, 6571)], 6723: [(5592, 8)], 5592: [(2412, 9), (5593, 6528)], 2425: [(6723, 10)], 2413: [(8020, 11)], 2414: [(2445, 12)], 2033: [(2416, 13)], 2435: [(2415, 14)], 2416: [(2435, 15), (2417, 16)], 1866: [(2033, 17), (2033, 37), (2432, 38)], 2439: [(1865, 18), (2419, 45)], 2418: [(2439, 19), (5704, 22)], 1865: [(2419, 20), (2454, 10078)], 2420: [(5704, 21), (2034, 47)], 2422: [(2421, 23)], 2423: [(2422, 24)], 2424: [(2423, 25), (2413, 27)], 2031: [(2424, 26), (5698, 79)], 2361: [(2425, 28)], 2426: [(6725, 29)], 2427: [(6725, 30)], 2428: [(2427, 31)], 2429: [(2428, 32), (2032, 33)], 2430: [(2030, 34)], 2431: [(2430, 35)], 2032: [(2431, 36), (2414, 10076)], 2433: [(2434, 39)], 2432: [(2437, 40)], 2437: [(2435, 41), (2436, 42)], 2436: [(2433, 43)], 2434: [(2438

In [122]:
np.unravel_index(bus_prop_mat.argmax(), bus_prop_mat.shape)

(739, 1598)

In [126]:
graph_reverse[1598]

[(1600, 1587)]

In [119]:
np.nonzero(bus_prop_mat == max(max(bus_prop_mat)))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [113]:
avg_carbon_emission_rate_node = carbon_emission @ bus_prop_mat

In [133]:
df_ = pd.read_csv("GIS/CATS_buses.csv")
df_.insert(2, 'power_demand', power_demand)
df_.to_csv('GIS/CATS_buses_input.csv', index=False)